<h1 style = 'text-align:center'>The Battle of Neighborhoods: Final Notebook</h1>

<h6>Install packages and libraries</h6>

In [1]:
!pip install bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup

!pip install wikipedia
import wikipedia as wp


In [2]:
import numpy as np
import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



<h3>1. Get the Coordinates of Hokkaido</h3>

In [3]:
address = 'Hokkaido'

geolocator = Nominatim(user_agent = "jap_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Hokkaido are {}, {}.".format(latitude, longitude))

The geographical coordinates of Hokkaido are 43.4519831, 142.8197834.


In [4]:
#Create map
map_Hokkaido = folium.Map(location = [latitude, longitude], zoom_start = 5)

#Add markers
label = 'Hokkaido'
label = folium.Popup(label, parse_html = True)
folium.CircleMarker(
    [latitude, longitude],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_Hokkaido)

map_Hokkaido

<h3>2. Determine the Latitude and Longitude of the Capital of Hokkaido</h3>

In [5]:
address = 'Sapporo'

geolocator = Nominatim(user_agent = "jap_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the capital of Hokkaido, Sapporo are {}, {}.".format(latitude, longitude))

The geographical coordinates of the capital of Hokkaido, Sapporo are 43.061936, 141.3542924.


In [6]:
#Create map
map_Sapporo = folium.Map(location = [latitude, longitude], zoom_start = 8)

#Add markers
label = 'Sapporo'
label = folium.Popup(label, parse_html = True)
folium.CircleMarker(
    [latitude, longitude],
    radius = 7,
    popup = label,
    color = 'red',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_Sapporo)

map_Sapporo

<h2>Get Information about Subprefectures</h2>

In [7]:
#Get the html source
html = requests.get('https://en.wikipedia.org/wiki/Subprefectures_of_Hokkaido').text

soup = BeautifulSoup(html,'lxml')
table = soup.find('table',{'class':'wikitable'})

In [8]:
Hokkaido=pd.read_html(str(table))
# convert list to dataframe
Hokkaido =pd.DataFrame(Hokkaido[0])
Hokkaido.columns = ['Subprefecture Index','Subprefecture',
                        'Japanese',
                        'Capital',
                        'Largest Municipality',
                        'Pop (2009)',
                        'Area',
                        'Cities', 'Towns', 'Villages']

Hokkaido = Hokkaido.drop([0,1,16]).set_index('Subprefecture Index', inplace=True)
Hokkaido

<h3>3. Select a Subprefecture and Find its Capital</h3>

In [9]:
address = 'Kamikawa'

geolocator = Nominatim(user_agent = "jap_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the subprefecture, Kamikawa are {}, {}.".format(latitude, longitude))

The geographical coordinates of the subprefecture, Kamikawa are 36.2134738, 139.1019026.


In [10]:
#Create map
map_Kamikawa = folium.Map(location = [latitude, longitude], zoom_start = 10)

#Add markers
label = 'Kamikawa'
label = folium.Popup(label, parse_html = True)
folium.CircleMarker(
    [latitude, longitude],
    radius = 7,
    popup = label,
    color = 'purple',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_Kamikawa)

map_Kamikawa

In [11]:
address = 'Asahikawa'

geolocator = Nominatim(user_agent = "jap_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of the subprefecture's capital, Asahikawa are {}, {}.".format(latitude, longitude))

The geographical coordinates of the subprefecture's capital, Asahikawa are 43.7627501, 142.3579263.


<h2 style = 'text-align:center'>Using Foursquare API<h2>

Credentials

In [12]:
CLIENT_ID = 'T4LGYBRGJL0K1JUIZKMPN4ZYD0F1GQTZC51EVIWUNDEOQOM2'
CLIENT_SECRET = '4CJLJJP2TLQVX12AJVOA3RNQGQQ0WUHZTLJKG5OVYKW245KJ'
VERSION = '20210412'

<h3>4. Collect Data on Places Around Asahikawa</h3>

In [13]:
# reading radius from user
radius = 70000
#reading search limit from user
Limit = 100
url  = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, Limit)
results = requests.get(url).json()
items = results['response']['groups'][0]['items']

In [14]:
#fetching omly relevant data from categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#flatten JSON, Normalize JSON to Dataframe
dataframe = json_normalize(items) 

# filter columns,consider only required columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
nearby = dataframe.loc[:, filtered_columns]

# filter the category for each row
nearby['venue.categories'] = nearby.apply(get_category_type, axis=1)

# clean columns
nearby.columns = [col.split('.')[-1] for col in nearby.columns]

#replce NaN values with Not found in address
nearby['address'] = nearby['address'].fillna("Not found")
nearby.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Jingisukan Daikokuya (成吉思汗 大黒屋 5丁目支店),BBQ Joint,4条通5,43.769385,142.357189,"[{'label': 'display', 'lat': 43.76938453207933...",740,070-0034,JP,旭川市,北海道,日本,"[4条通5, 旭川市, 北海道, 070-0034, 日本]",NaN,NaN,4d43e321befe236abfe9eee2
1,Coach & Four (コーチャンフォー 旭川店),Bookstore,宮前1条2-4-1,43.759140,142.368168,"[{'label': 'display', 'lat': 43.75914009518399...",916,078-8328,JP,旭川市,北海道,日本,"[宮前1条2-4-1, 旭川市, 北海道, 078-8328, 日本]",NaN,NaN,4c8db2181992a1cdfba8d7fb
2,Dokushaku Sanshiro (独酌三四郎),Sake Bar,2条通5-左7,43.768156,142.356412,"[{'label': 'display', 'lat': 43.76815566195045...",613,070-0032,JP,旭川市,北海道,日本,"[2条通5-左7, 旭川市, 北海道, 070-0032, 日本]",NaN,NaN,4d5a5ce356f2b60c60da822f
3,回転寿しトリトン 旭神店,Sushi Restaurant,旭神3条5-1,43.734625,142.389345,"[{'label': 'display', 'lat': 43.73462525500854...",4023,078-8308,JP,旭川市,北海道,日本,"[旭神3条5-1 (Toriton), 旭川市, 北海道, 078-8308, 日本]",Toriton,NaN,4b5983f6f964a520c98a28e3
4,Dormy Inn Asahikawa (ドーミーイン旭川),Hotel,5条通6-964-1,43.770982,142.359855,"[{'label': 'display', 'lat': 43.77098154369040...",929,070-0035,JP,旭川市,北海道,日本,"[5条通6-964-1, 旭川市, 北海道, 070-0035, 日本]",NaN,NaN,4dbc084b6e810768bf52d5bd
5,Asahiyama Zoo (旭山動物園),Zoo,東旭川町倉沼,43.768274,142.478873,"[{'label': 'display', 'lat': 43.76827370036577...",9742,078-8205,JP,旭川市,北海道,日本,"[東旭川町倉沼, 旭川市, 北海道, 078-8205, 日本]",NaN,NaN,4b5a4ef5f964a520f5bb28e3
6,みよしの 旭川大雪通店,Dumpling Restaurant,4条通19-1717-2,43.764032,142.377853,"[{'label': 'display', 'lat': 43.76403161463792...",1608,NaN,JP,旭川市,北海道,日本,"[4条通19-1717-2, 旭川市, 北海道, 日本]",NaN,NaN,4ff55a16e4b036acfa0b285c
7,Polar Bear Museum (ほっきょくぐま館),Zoo Exhibit,東旭川町倉沼,43.768392,142.479166,"[{'label': 'display', 'lat': 43.76839207025719...",9766,078-8205,JP,旭川市,北海道,日本,"[東旭川町倉沼 (旭山動物園), 旭川市, 北海道, 078-8205, 日本]",旭山動物園,NaN,4c9edcbe8afca09310690516
8,AEON Mall (イオンモール旭川駅前),Shopping Mall,宮下通7-2-5,43.764055,142.356876,"[{'label': 'display', 'lat': 43.764055, 'lng':...",168,070-0030,JP,旭川市,北海道,日本,"[宮下通7-2-5, 旭川市, 北海道, 070-0030, 日本]",NaN,NaN,53c75fb5498ef985d0027ba3
9,自由軒,Yoshoku Restaurant,5条通8,43.770131,142.361925,"[{'label': 'display', 'lat': 43.77013067133397...",882,070-0035,JP,旭川市,北海道,日本,"[5条通8, 旭川市, 北海道, 070-0035, 日本]",NaN,NaN,4d9d2fb57509b1f73c3f3ca0


In [15]:
Asahikawa = nearby
Asahikawa = Asahikawa.drop(['address','labeledLatLngs','distance','postalCode','cc','state','country', 'formattedAddress','crossStreet','id'], axis = 1)
Asahikawa

fixed_columns = [Asahikawa.columns[-1]] + list(Asahikawa.columns[:-1])
Asahikawa = Asahikawa[fixed_columns]
Asahikawa

,neighborhood,name,categories,lat,lng,city
0,NaN,Jingisukan Daikokuya (成吉思汗 大黒屋 5丁目支店),BBQ Joint,43.769385,142.357189,旭川市
1,NaN,Coach & Four (コーチャンフォー 旭川店),Bookstore,43.759140,142.368168,旭川市
2,NaN,Dokushaku Sanshiro (独酌三四郎),Sake Bar,43.768156,142.356412,旭川市
3,NaN,回転寿しトリトン 旭神店,Sushi Restaurant,43.734625,142.389345,旭川市
4,NaN,Dormy Inn Asahikawa (ドーミーイン旭川),Hotel,43.770982,142.359855,旭川市
...,...,...,...,...,...,...
95,NaN,Kumagera (くまげら),Japanese Restaurant,43.345039,142.391110,富良野市
96,NaN,New Furano Prince Hotel (新富良野プリンスホテル),Hotel,43.323173,142.354495,富良野市
97,NaN,回転寿司 トピカル,Sushi Restaurant,43.344191,142.385034,富良野
98,NaN,FB FURANO BURGER KICHEN&SHOP,Burger Joint,43.342775,142.436228,富良野市


In [16]:
# create map of New York using latitude and longitude values
map_Asahikawa = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, neighborhood in zip(Asahikawa['lat'], Asahikawa['lng'], Asahikawa['city'], Asahikawa['name']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Asahikawa)  
    
map_Asahikawa


In [17]:
# one hot encoding
Asahikawa_onehot = pd.get_dummies(Asahikawa[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Asahikawa_onehot['city'] = Asahikawa['city'] 

# move neighborhood column to the first column
fixed_columns = [Asahikawa_onehot.columns[-1]] + list(Asahikawa_onehot.columns[:-1])
Asahikawa_onehot = Asahikawa_onehot[fixed_columns]

Asahikawa_onehot.head()

,city,Art Gallery,BBQ Joint,Bakery,Bath House,Bed & Breakfast,Bookstore,Brewery,Burger Joint,Café,...,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Train Station,Tree,Waterfall,Yakitori Restaurant,Yoshoku Restaurant,Zoo,Zoo Exhibit
0,旭川市,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,旭川市,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,旭川市,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,旭川市,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,旭川市,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
Asahikawa_grouped = Asahikawa_onehot.groupby('city').mean().reset_index()
Asahikawa_grouped

,city,Art Gallery,BBQ Joint,Bakery,Bath House,Bed & Breakfast,Bookstore,Brewery,Burger Joint,Café,...,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Train Station,Tree,Waterfall,Yakitori Restaurant,Yoshoku Restaurant,Zoo,Zoo Exhibit
0,Higashikawa-chō,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,上富良野町,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,中富良野町,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,北竜町,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,富良野,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,富良野市,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,...,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,旭川市,0.000000,0.052632,0.000000,0.0,0.017544,0.017544,0.017544,0.017544,0.017544,...,0.017544,0.000000,0.035088,0.017544,0.000000,0.000000,0.017544,0.035088,0.017544,0.192982
7,東神楽町,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,比布町,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,滝川市,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


<h5>The top 5 most common venues</h5>

In [19]:
num_top_venues = 5

for hood in Asahikawa_grouped['city']:
    print("----"+hood+"----")
    temp = Asahikawa_grouped[Asahikawa_grouped['city'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Higashikawa-chō----
                 venue  freq
0             Mountain  0.25
1                Hotel  0.25
2            Rest Area  0.25
3  Sporting Goods Shop  0.25
4          Art Gallery  0.00


----上富良野町----
                 venue  freq
0      Soba Restaurant   0.5
1                  Inn   0.5
2  Japanese Restaurant   0.0
3             Mountain   0.0
4               Museum   0.0


----中富良野町----
                 venue  freq
0                 Farm   1.0
1          Art Gallery   0.0
2  Japanese Restaurant   0.0
3             Mountain   0.0
4               Museum   0.0


----北竜町----
                 venue  freq
0            Rest Area   0.5
1               Garden   0.5
2          Art Gallery   0.0
3  Japanese Restaurant   0.0
4             Mountain   0.0


----富良野----
              venue  freq
0  Sushi Restaurant   1.0
1       Art Gallery   0.0
2   Soba Restaurant   0.0
3          Mountain   0.0
4            Museum   0.0


----富良野市----
                       venue  freq
0  Japanese Cu

<h3>5. Find the top 10 venues.</h3>

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Asahikawa_venues_sorted = pd.DataFrame(columns=columns)
Asahikawa_venues_sorted['city'] = Asahikawa_grouped['city']

for ind in np.arange(Asahikawa_grouped.shape[0]):
    Asahikawa_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Asahikawa_grouped.iloc[ind, :], num_top_venues)

Asahikawa_venues_sorted.head(10)

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Higashikawa-chō,Hotel,Mountain,Sporting Goods Shop,Rest Area,Zoo Exhibit,Cupcake Shop,Hot Spring,Hill,Garden,Furniture / Home Store
1,上富良野町,Inn,Soba Restaurant,Cupcake Shop,Ice Cream Shop,Hotel,Hot Spring,Hill,Garden,Furniture / Home Store,Farm
2,中富良野町,Farm,Zoo Exhibit,Cupcake Shop,Ice Cream Shop,Hotel,Hot Spring,Hill,Garden,Furniture / Home Store,Dumpling Restaurant
3,北竜町,Garden,Rest Area,Zoo Exhibit,Italian Restaurant,Ice Cream Shop,Hotel,Hot Spring,Hill,Furniture / Home Store,Farm
4,富良野,Sushi Restaurant,Zoo Exhibit,Cupcake Shop,Ice Cream Shop,Hotel,Hot Spring,Hill,Garden,Furniture / Home Store,Farm
5,富良野市,Japanese Curry Restaurant,Cupcake Shop,Japanese Restaurant,Hotel,Steakhouse,Burger Joint,Dessert Shop,Hot Spring,Hill,Garden
6,旭川市,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo
7,東神楽町,Ice Cream Shop,Hot Spring,Discount Store,Zoo Exhibit,Cupcake Shop,Hotel,Hill,Garden,Furniture / Home Store,Farm
8,比布町,Ski Area,Bath House,Cupcake Shop,Ice Cream Shop,Hotel,Hot Spring,Hill,Garden,Furniture / Home Store,Farm
9,滝川市,BBQ Joint,Zoo Exhibit,Cupcake Shop,Ice Cream Shop,Hotel,Hot Spring,Hill,Garden,Furniture / Home Store,Farm


<h2>Clustering Data</h2>

In [22]:
# set number of clusters
kclusters = 5

Asahikawa_clustering = Asahikawa_grouped.drop(['city'],1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Asahikawa_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 1, 4, 1, 1, 1, 1, 3])

In [23]:
# add clustering labels
Asahikawa_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Asahikawa_merged = Asahikawa
# merge Asahikawa_grouped with Asahikawa to add latitude/longitude for each neighborhood
Asahikawa_merged = Asahikawa_merged.join(Asahikawa_venues_sorted.set_index('city'), on='city')

Asahikawa_merged['Cluster Labels'] = Asahikawa_merged['Cluster Labels'].fillna(0.0).astype(int)['Cluster Labels'] = Asahikawa_merged['Cluster Labels'].fillna(0.0).astype(int)
Asahikawa_merged.head() # check the last columns!

,neighborhood,name,categories,lat,lng,city,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NaN,Jingisukan Daikokuya (成吉思汗 大黒屋 5丁目支店),BBQ Joint,43.769385,142.357189,旭川市,1,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo
1,NaN,Coach & Four (コーチャンフォー 旭川店),Bookstore,43.759140,142.368168,旭川市,1,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo
2,NaN,Dokushaku Sanshiro (独酌三四郎),Sake Bar,43.768156,142.356412,旭川市,1,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo
3,NaN,回転寿しトリトン 旭神店,Sushi Restaurant,43.734625,142.389345,旭川市,1,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo
4,NaN,Dormy Inn Asahikawa (ドーミーイン旭川),Hotel,43.770982,142.359855,旭川市,1,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo


<h3>Cluster Map</h3>

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Asahikawa_merged['lat'], Asahikawa_merged['lng'],
                                  Asahikawa_merged['city'], Asahikawa_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Analzying Clusters</h3>

In [25]:
Asahikawa_merged.loc[Asahikawa_merged['Cluster Labels'] == 0, 
                    Asahikawa_merged.columns[[1] + list(range(5, Asahikawa_merged.shape[1]))]]

,name,city,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,北の住まい設計社,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,Farm Tomita (ファーム富田),中富良野町,0,Farm,Zoo Exhibit,Cupcake Shop,Ice Cream Shop,Hotel,Hot Spring,Hill,Garden,Furniture / Home Store,Dumpling Restaurant
87,La Vista Taisetsuzan (ラビスタ大雪山 La Vista Daisets...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
Asahikawa_merged.loc[Asahikawa_merged['Cluster Labels'] == 1, 
                    Asahikawa_merged.columns[[1] + list(range(5, Asahikawa_merged.shape[1]))]]

,name,city,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Jingisukan Daikokuya (成吉思汗 大黒屋 5丁目支店),旭川市,1,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo
1,Coach & Four (コーチャンフォー 旭川店),旭川市,1,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo
2,Dokushaku Sanshiro (独酌三四郎),旭川市,1,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo
3,回転寿しトリトン 旭神店,旭川市,1,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo
4,Dormy Inn Asahikawa (ドーミーイン旭川),旭川市,1,Zoo Exhibit,Ramen Restaurant,Hotel,BBQ Joint,Yoshoku Restaurant,Sushi Restaurant,Sake Bar,Coffee Shop,Dumpling Restaurant,Zoo
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Sanai-no-Oka Observatory Park (三愛の丘展望公園),美瑛町,1,Tree,Park,Hill,Farm,Art Gallery,Café,Yoshoku Restaurant,Waterfall,Train Station,Bakery
94,手打ち蕎麦 まん作,上富良野町,1,Inn,Soba Restaurant,Cupcake Shop,Ice Cream Shop,Hotel,Hot Spring,Hill,Garden,Furniture / Home Store,Farm
95,Kumagera (くまげら),富良野市,1,Japanese Curry Restaurant,Cupcake Shop,Japanese Restaurant,Hotel,Steakhouse,Burger Joint,Dessert Shop,Hot Spring,Hill,Garden
96,New Furano Prince Hotel (新富良野プリンスホテル),富良野市,1,Japanese Curry Restaurant,Cupcake Shop,Japanese Restaurant,Hotel,Steakhouse,Burger Joint,Dessert Shop,Hot Spring,Hill,Garden


In [27]:
Asahikawa_merged.loc[Asahikawa_merged['Cluster Labels'] == 2, 
                    Asahikawa_merged.columns[[1] + list(range(5, Asahikawa_merged.shape[1]))]]

,name,city,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,ファームレストラン リヴィスタ,砂川市,2,Italian Restaurant,Zoo Exhibit,Cupcake Shop,Ice Cream Shop,Hotel,Hot Spring,Hill,Garden,Furniture / Home Store,Farm


In [28]:
Asahikawa_merged.loc[Asahikawa_merged['Cluster Labels'] == 3, 
                    Asahikawa_merged.columns[[1] + list(range(5, Asahikawa_merged.shape[1]))]]

,name,city,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
99,松尾ジンギスカン 本店,滝川市,3,BBQ Joint,Zoo Exhibit,Cupcake Shop,Ice Cream Shop,Hotel,Hot Spring,Hill,Garden,Furniture / Home Store,Farm


In [29]:
Asahikawa_merged.loc[Asahikawa_merged['Cluster Labels'] == 4, 
                    Asahikawa_merged.columns[[1] + list(range(5, Asahikawa_merged.shape[1]))]]

,name,city,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,回転寿司 トピカル,富良野,4,Sushi Restaurant,Zoo Exhibit,Cupcake Shop,Ice Cream Shop,Hotel,Hot Spring,Hill,Garden,Furniture / Home Store,Farm


<h4>Get category information of data</h4>

In [30]:
Asahikawa.head(10)

,neighborhood,name,categories,lat,lng,city
0,NaN,Jingisukan Daikokuya (成吉思汗 大黒屋 5丁目支店),BBQ Joint,43.769385,142.357189,旭川市
1,NaN,Coach & Four (コーチャンフォー 旭川店),Bookstore,43.759140,142.368168,旭川市
2,NaN,Dokushaku Sanshiro (独酌三四郎),Sake Bar,43.768156,142.356412,旭川市
3,NaN,回転寿しトリトン 旭神店,Sushi Restaurant,43.734625,142.389345,旭川市
4,NaN,Dormy Inn Asahikawa (ドーミーイン旭川),Hotel,43.770982,142.359855,旭川市
5,NaN,Asahiyama Zoo (旭山動物園),Zoo,43.768274,142.478873,旭川市
6,NaN,みよしの 旭川大雪通店,Dumpling Restaurant,43.764032,142.377853,旭川市
7,NaN,Polar Bear Museum (ほっきょくぐま館),Zoo Exhibit,43.768392,142.479166,旭川市
8,NaN,AEON Mall (イオンモール旭川駅前),Shopping Mall,43.764055,142.356876,旭川市
9,NaN,自由軒,Yoshoku Restaurant,43.770131,142.361925,旭川市


In [31]:
print('There are {} uniques categories.'.format(len(Asahikawa['categories'].unique())))

There are 51 uniques categories.
